<h1> 개인과제 </h1>
<h2> 주피터랩에서 데이터를 입력받으면 3가지 DB에 동시에 CRUD가 되는 시스템을 만들어라.</h2>
<h3> 예) input을 이용하여 홍길동(엔터), 33(엔터), <전화번호> 111-2222(엔터), <주소> 해운대구 </h3>
<h3>입력하고, 이름을 고르고 수정하고, 삭제 가능하게 하라</h3>

In [24]:
# !pip install mysql-connector-python
# !pip install pymongo
# !pip install oracledb

In [79]:
import mysql.connector
class DBSystem:
    def __init__(self):
        import mysql.connector
        from pymongo import MongoClient
        import oracledb
        str= """
            1. 데이터 등록
            2. 데이터 읽기
            3. 데이터 수정
            4. 데이터 삭제
            5. 종료
            고르세요
            """
        inp =1
        MYSQL()
        MONGODB()
        ORACLE()
        
        while True:
            print(str)
            inp = input("선택하세요:")
            print(inp,"고르셨네요")
            # 데이터 등록
            if inp == '1':
                self.inp_name = input("이름을 입력하세요:")
                self.inp_age = input("나이를 입력하세요:")
                self.inp_address = input("주소를 입력하세요:")
                Create()
                print(self.inp_name)
                
            if inp == '2':
                self.inp_name = input("찾고싶은 데이터의 name을 입력하세요")
                Read()
                print(self.inp_name)
                
            if inp == '3':
                self.inp_update = input("변경하고 싶은 데이터의 name을 입력하세요")
                self.inp_data = input("변경하고자 하는 데이터의 종류를 입력하세요(name,age,address)")
                self.inp_change = input("변경할 데이터를 입력하세요")
                Update()
                print(self.inp_name)
            if inp == '4':
                self.inp_name = input("삭제하고 싶은 데이터의 name을 입력하세요")
                inp_choose = input("삭제하시겠습니까? (네, 아니요)")
                if inp_choose == "네":
                    Delete()
                else:
                    print("다시")
                    print(self.inp_name)
                
            if inp == '5':
                print("종료합니다.")
                break
    
    def MYSQL():
        global conn_sql, cursor_sql
        conn_sql = mysql.connector.connect(
            host="mysql",
            user="root",
            password="1234",
            database="judb"
        )
        cursor_sql = conn_sql.cursor()
    
    def MONGODB():
        global collection
        client = MongoClient("mongodb://mongodb:27017")
        db = client["mdb"]
        collection = db["c1"]
    
    def ORACLE():
        global conn_ora, cursor_ora
        try:
            conn_ora = oracledb.connect(
                user="pknu",
                password="1234",
                dsn = "oracle/XEPDB1"
            )
            cursor_ora = conn_ora.cursor()
        except Exception as e:
            print("Oracle DB 연결 실패:", e)
    
    def Create(self):
        sql = "INSERT INTO users(name, age, city) VALUES (%s, %s, %s)"
        values = (f"{self.inp_name}",self.inp_age, f"{self.inp_address}")
        cursor_sql.execute(sql, values)
        conn_sql.commit()
        print("MySQL 데이터 삽입 완료")
    
        user = {"name":f"{self.inp_name}","age":self.inp_age,"city":f"{self.inp_address}"}
        insert_result = collection.insert_one(user)
        print("mongoDB 데이터 삽입 완료")
    
        
        que = "INSERT INTO students (name,age,email) VALUES (:1,:2,:3)"
        values = (f"{self.inp_name}",self.inp_age, f"{self.inp_address}")
        cursor_ora.execute(que,values)
        conn_ora.commit()
        print("Oracle 데이터 삽입 완료")
    
    def Read(self):
        que = "SELECT * FROM users"
        filters = " WHERE name = %s"
        value = (f"{self.inp_name}",)
        cursor_sql.execute(que + filters,value)
        filtered = cursor_sql.fetchall()
        print("======================================================")
        print(self.inp_name)
        print("MySQL에서 조회한 정보 출력")
        for f in filtered:
            if f == " ":
                print("MySQL에서는 조회할 정보가 없어요")
            else:
                print(f)
        print("======================================================")
        print("mongoDB에서 조회한 정보 출력")
        user = collection.find_one({"name":f"{self.inp_name}"},{"_id":0})
        if user == None:
            print("mongoDB에서는 조회할 정보가 없어요")
        else:
            print(user)
        print("======================================================")
        que = "SELECT * FROM students WHERE name = :name"
        cursor_ora.execute(que, {'name':f"{self.inp_name}"})
        print("Oracle에서 조회한 정보 출력")
        users = cursor_ora.fetchall()
        for user in users:
            if user == "none":
                print("Oracle에서는 조회할 정보가 없어요")
            else:
                print(user)
    
    def Update(self):
        if self.inp_data == "age":
            sql = "UPDATE users SET age = %s WHERE name = %s"
        elif self.inp_data == "name":
            sql = "UPDATE users SET name = %s WHERE name = %s"
        elif self.inp_data == "address":
            sql = "UPDATE users SET city = %s WHERE name = %s"
        else:
            print("잘못된 항목입니다.")
            return       
        values = (f"{self.inp_change}",f"{self.inp_name}")
        cursor_sql.execute(sql, values)
        conn_sql.commit()
        print("MySQL 수정완료")
    
        if self.inp_data == "address":
            update = collection.update_many({"name":f"{self.inp_name}"},{"$set":{"city":f"{self.inp_change}"}})
        elif self.inp_data == "name":
            update = collection.update_many({"name":f"{self.inp_name}"},{"$set":{"name":f"{self.inp_change}"}})
        elif self.inp_data == "age":
            update = collection.update_many({"name":f"{self.inp_name}"},{"$set":{"age":f"{self.inp_change}"}})
        else:
            print("잘못된 항목입니다.")
            return    
        print("mongoDB 수정완료")
    
    
        if self.inp_data == "address":
            que = "UPDATE students SET email = :1 WHERE name = :2"
        elif self.inp_data == "name":
            que = "UPDATE students SET name = :1 WHERE name = :2"
        elif self.inp_data == "age":
            que = "UPDATE students SET age = :1 WHERE name = :2"
        else:
            print("잘못된 항목입니다.")
            return    
            
        values = (f"{self.inp_change}",f"{self.inp_name}")
        cursor_ora.execute(que, values)
        conn_ora.commit()
        print("Oracle 수정완료")    
    
    def Delete(self):
        que = "DELETE FROM users WHERE name = %s"
        cursor_sql.execute(que, (f"{self.inp_name}",))
        conn_sql.commit()
        print(f"MySQL {self.inp_name} 삭제완료")
    
        delete = collection.delete_one({"name":f"{self.inp_name}"})
        print(f"mongoDB {self.inp_name} 삭제완료")
    
        que = "DELETE FROM students WHERE name = :v_name"
        cursor_ora.execute(que, {"v_name":f"{self.inp_name}"})
        conn_ora.commit()
        print(f"Oracle {self.inp_name} 삭제완료")

In [80]:
print(DBSystem())


            1. 데이터 등록
            2. 데이터 읽기
            3. 데이터 수정
            4. 데이터 삭제
            5. 종료
            고르세요
            


선택하세요: 2


2 고르셨네요


찾고싶은 데이터의 name을 입력하세요 이성훈


MySQL에서 조회한 정보 출력
(16, 'ㅇ아아아아앙아아', 90000, 'Incheon')
mongoDB에서 조회한 정보 출력
{'name': 'ㅇ아아아아앙아아', 'age': '90000', 'city': 'Incheon'}
Oracle에서 조회한 정보 출력
(12, 'ㅇ아아아아앙아아', 90000, 'Incheon')
이성훈

            1. 데이터 등록
            2. 데이터 읽기
            3. 데이터 수정
            4. 데이터 삭제
            5. 종료
            고르세요
            


선택하세요: 5


5 고르셨네요
종료합니다.
